In [1]:
import librosa.display
import librosa as lb
import os
import numpy as np
import pickle
import matplotlib.pyplot as plt
import time
import multiprocessing
import itertools
import sys
from collections import OrderedDict
from  more_itertools import unique_everseen
from scipy.stats import skew
from scipy.stats import kurtosis

The main flow of this code is further down the notebook. It requires a number of pre-defined functions and we could have saved all of these into a library and imported them, but just to be explicit they're presented here.

First up, we'll need a function to split a song up into time chunks:

In [2]:
# a function to split up a song into TIME chunks
def splitT(mint,maxt,songdat):
    splittime=[]
    for i in range(mint,maxt):
        splittime.append(songdat[:,i]) # first axis is freq, second axis is time. Return all freq for specific time range.
    return (np.array(splittime))

...and we'll need a function to split a song up into frequency chunks:

In [3]:
# a function to split up a song into FREQ chunks
def splitF(minv, maxv, songdat):
    splitfreq = []
    for i in range(minv,maxv):
        splitfreq.append(songdat[i,:]) # first axis is freq, second axis is time. Return all time for specific freq range.
    #print(splitfreq)
    return (np.array(splitfreq))

We'll need a master function to extract the machine learning features. This is the main function which gets features from the songs. Most values returned are the mean of the whole time series, hence '_a'.

In [137]:
def get_features_mean(song,sr,hop_length,n_fft):
#     try:
    print('>>> extracting features...')
    y_harmonic, y_percussive = lb.effects.hpss(song) #split song into harmonic and percussive parts
    stft_harmonic=lb.core.stft(y_harmonic, n_fft=n_fft, hop_length=hop_length)	#Compute power spectrogram.
    stft_percussive=lb.core.stft(y_percussive, n_fft=n_fft, hop_length=hop_length)	#Compute power spectrogram.
    #stft_all=lb.core.stft(song, n_fft=n_fft, hop_length=hop_length)	#Compute power spectrogram.
    band_resolution=[5] #[5,25] Choose number of bands, do low and high resolution?
    bands_dict=OrderedDict()

    for no_bands in band_resolution:
        bands=np.logspace(1.3,4,no_bands)/10 #note that as n_fft is 2050 (I've decided this is sensible resolution), bands/10=freq
        bands_int=bands.astype(int)
        bands_int_unique=list(unique_everseen(bands_int)) #removing double entries less than 100Hz, because logspace bunches up down there and we don't need doubles when rounding to the nearest 10 Hz.

        for i in range(0,len(bands_int_unique)-1):

            _h=lb.feature.rms(y=(splitF(bands_int_unique[i],bands_int_unique[i+1],stft_harmonic)))
            _p=lb.feature.rms(y=(splitF(bands_int_unique[i],bands_int_unique[i+1],stft_percussive)))

            #Calculate statistics for harmoinc and percussive over the time series.
            rms_h=np.mean(np.abs(_h))
            std_h=np.std(np.abs(_h))
            skew_h=skew(np.mean(np.abs(_h), axis=0))  #skew of the time series (avg along freq axis, axis=0)
            kurtosis_h=kurtosis(np.mean(np.abs(_h), axis=0), fisher=True, bias=True) #kurtosis of time series (avg along freq axis=0)

            rms_p=np.mean(np.abs(_p))
            std_p=np.std(np.abs(_p))
            skew_p=skew(np.mean(np.abs(_p), axis=0))  #skew of the time series (avg along freq axis, axis=0)
            kurtosis_p=kurtosis(np.mean(np.abs(_p), axis=0), fisher=True, bias=True) #kurtosis of time series (avg along freq axis=0)

            #Append results to dict, with numbers as band labels
            bands_dict.update({'{0}band_rms_h{1}'.format(no_bands,i):rms_h,'{0}band_rms_p{1}'.format(no_bands,i):rms_p})
            bands_dict.update({'{0}band_std_h{1}'.format(no_bands,i):std_h,'{0}band_std_p{1}'.format(no_bands,i):std_p})
            bands_dict.update({'{0}band_skew_h{1}'.format(no_bands,i):skew_h,'{0}band_skew_p{1}'.format(no_bands,i):skew_p})
            bands_dict.update({'{0}band_kurtosis_h{1}'.format(no_bands,i):kurtosis_h,'{0}band_kurtosis_p{1}'.format(no_bands,i):kurtosis_p})

    #stft=lb.feature.chroma_stft(song, sr, n_fft=n_fft, hop_length=hop_length)	#Compute a chromagram from a waveform or power spectrogram.
    #stft_a=np.mean(stft[0])
    #stft_std=np.std(stft[0])
    #rmse=lb.feature.rms(y=song)	#Compute root-mean-square (RMS) energy for each frame, either from the audio samples y or from a spectrogram S.
    #rmse_a=np.mean(rmse)
    #rmse_std=np.std(rmse)
    rmseH=np.abs(lb.feature.rms(y=stft_harmonic))	#Compute root-mean-square (RMS) energy for harmonic
    rmseH_a=np.mean(rmseH)
    rmseH_std=np.std(rmseH)
    rmseH_skew=skew(np.mean(rmseH, axis=0))
    rmseH_kurtosis=kurtosis(np.mean(rmseH, axis=0), fisher=True, bias=True)

    rmseP=np.abs(lb.feature.rms(y=stft_percussive))	#Compute root-mean-square (RMS) energy for percussive
    rmseP_a=np.mean(rmseP)
    rmseP_std=np.std(rmseP)
    rmseP_skew=skew(np.mean(rmseP, axis=0))
    rmseP_kurtosis=kurtosis(np.mean(rmseP, axis=0), fisher=True, bias=True)

    centroid=lb.feature.spectral_centroid(song, sr, n_fft=n_fft, hop_length=hop_length)	#Compute the spectral centroid.
    centroid_a=np.mean(centroid)
    centroid_std=np.std(centroid)
    bw=lb.feature.spectral_bandwidth(song, sr, n_fft=n_fft, hop_length=hop_length)	#Compute p’th-order spectral bandwidth:
    bw_a=np.mean(bw)
    bw_std=np.std(bw)
    contrast=lb.feature.spectral_contrast(song, sr, n_fft=n_fft, hop_length=hop_length)	#Compute spectral contrast [R16]
    contrast_a=np.mean(contrast)
    contrast_std=np.std(contrast)
    polyfeat=lb.feature.poly_features(y_harmonic, sr, n_fft=n_fft, hop_length=hop_length)	#Get coefficients of fitting an nth-order polynomial to the columns of a spectrogram.
    polyfeat_a=np.mean(polyfeat[0])
    polyfeat_std=np.std(polyfeat[0])
    tonnetz=lb.feature.tonnetz(librosa.effects.harmonic(y_harmonic), sr)	#Computes the tonal centroid features (tonnetz), following the method of [R17].
    tonnetz_a=np.mean(tonnetz)
    tonnetz_std=np.std(tonnetz)
    zcr=lb.feature.zero_crossing_rate(song, sr, hop_length=hop_length)  #zero crossing rate
    zcr_a=np.mean(zcr)
    zcr_std=np.std(zcr)
    onset_env=lb.onset.onset_strength(y_percussive, sr=sr)
    onset_a=np.mean(onset_env)
    onset_std=np.std(onset_env)
    D = librosa.stft(song)
    times = librosa.frames_to_time(np.arange(D.shape[1])) #not returned, but could be if you want to plot things as a time series
    bpm,beats=lb.beat.beat_track(y=y_percussive, sr=sr, onset_envelope=onset_env, units='time')
    beats_a=np.mean(beats)
    beats_std=np.std(beats)

    features_dict=OrderedDict({'rmseP_a':rmseP_a,'rmseP_std':rmseP_std,'rmseH_a':rmseH_a,'rmseH_std':rmseH_std,'centroid_a':centroid_a,'centroid_std':centroid_std,'bw_a':bw_a,'bw_std':bw_std,'contrast_a':contrast_a,'contrast_std':contrast_std,'polyfeat_a':polyfeat_a,'polyfeat_std':polyfeat_std,'tonnetz_a':tonnetz_a,'tonnetz_std':tonnetz_std,'zcr_a':zcr_a,'zcr_std':zcr_std,'onset_a':onset_a,'onset_std':onset_std,'bpm':bpm, 'rmseP_skew':rmseP_skew, 'rmseP_kurtosis':rmseP_kurtosis, 'rmseH_skew':rmseH_skew, 'rmseH_kurtosis':rmseH_kurtosis})
    combine_features={**features_dict,**bands_dict}
    print('>>> features extracted successfully')
    return combine_features

#     except:
#         print('.'*20+'FAILED'+'.'*20)
#         print('.'*40)

We'll need a function to load the songs into python:

In [138]:
# load music function, accepts any format i've encountered: mp3,wav,wma bla bla
def load_music(songname1,songpath1):
    try:
        print('loading the song: {0} ......... located here: {1} '.format(songname1, songpath1))
        songdata1, sr1 = lb.load(songpath1) # librosa library used to grab song data and sample rate
        print ('done........ '+songname1)
        return [songname1,songdata1,sr1]
    except: # the song could be corrupt or you could be trying to load something which isn't a song
        print('..............................FAILED...............................')
        print(songpath1)
        print('...................................................................')
        return

In [139]:
# function for saving the python dictionaries to disk
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [140]:
# function for loading the python dictionaries from disk
def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [141]:
# If you want a grid-plot to test anything out, this will help. 
# Although I've made sure get_features returns only averaged values, not time-series data, so meh.
def gridplot(data_dict,feature,size,N,ind):
    f, axarr = plt.subplots(size, size, sharey=True)
    i=0
    j=0
    for key in data_dict:
        #print (i,j)
        axarr[i,j].plot(np.convolve(data_dict[key][feature][ind],np.ones((N,))/N, mode='valid'))
        axarr[i, j].set_title(key[:3])
        if j==size-1: i+=1
        j=0 if j==size-1 else j+1
    for i in range(1,size,1):
        plt.setp([a.get_yticklabels() for a in axarr[:, i]], visible=False)
    plt.savefig('test.png')

-----

This is where we actually start to write the main program...

Let's make a note of the start time for loading our data:

In [142]:
start_load=time.time() # we're going to want know how long this takes...

This is a computationally expensive routine so we should use multiprocessing. Let's work out how many cores are available:

In [143]:
num_workers = multiprocessing.cpu_count() 
print('you have {0} cores available to do your bidding...'.format(num_workers))

you have 12 cores available to do your bidding...


We need to define some parameters:

In [144]:
n_fft1=2050 # important parameter here; this is the size of the fft window. these are sensible values
hop_length1=441 # n_fft/5 is a sensible value. Too large and you don't sample properly.

Now let's create a temporary database for the songs:

In [145]:
# create song database, songdb:
songname_tmp=[]
songpath_tmp=[]

Everything is set to start reading in the song names. So we need to tell the program where to find them, i.e. the name of the artist directory:

In [146]:
path='free_music'
filename = librosa.ex('trumpet')

The output feature file will have the same name with the suffix "_data":

In [147]:
savefile=str(path)+'_data'

We can get the list of songs by looping through the directory contents:

In [148]:
for song in os.listdir(path):
    print (song)
    songname_tmp.append(song)
    songpath_tmp.append(path+'/'+song)

AMBForst_Forest (ID 0100)_BSB.wav


In [149]:
songname=songname_tmp #i'm just reassigning the name incase of tests with commented out lines...
songpath=songpath_tmp

Loading the song data is a slow process, so we'll parallelize it over multiple processors. A **starmap** is a way to pass multiple arguments to a single function using multiple processors.

In [150]:
print('>>> loading songs...')

# =====
# Parallel version:
#with multiprocessing.Pool(processes=num_workers) as pool:
#    songdb=pool.starmap(load_music,zip(songname,songpath))
#    pool.close()
#    pool.join()

# Serial version:
songdb=[]
for i in range(0, len(songname)):
    print(songname[i])
    print(songpath[i])
    print()
    
    songdb.append(load_music(songname[i], songpath[i]))
# =====

print('>>> finished loading songs into songdb')

>>> loading songs...
AMBForst_Forest (ID 0100)_BSB.wav
free_music/AMBForst_Forest (ID 0100)_BSB.wav

loading the song: AMBForst_Forest (ID 0100)_BSB.wav ......... located here: free_music/AMBForst_Forest (ID 0100)_BSB.wav 
done........ AMBForst_Forest (ID 0100)_BSB.wav
>>> finished loading songs into songdb


Let's run a quick check to remove any entries where loading may have failed for any reason (rare cases):

In [151]:
print ('>>> loaded {0} songs into memory'.format(len(songdb)))
songdb=[x for x in songdb if x is not None] 

>>> loaded 1 songs into memory


In [152]:
song_name=[] # text
song_data=[] # list of numbers
song_sr=[]   # sample rate

for song1 in songdb: 
    song_name.append(song1[0])
    song_data.append(song1[1])
    song_sr.append(song1[2])

That's the last step in loading the songs, so let's check how long it took:

In [153]:
stop_load=time.time()

loadtime = stop_load - start_load
print('>>> loading the songs into memory took {0} seconds'.format(loadtime))

>>> loading the songs into memory took 2.7637243270874023 seconds


Now we'll extract the machine learning features from the songs, so let's make a note of our start time for this step:

In [154]:
start_feat = time.time() # note the time

To extract the features we use the **get_features_mean** function defined earlier. Again this can be slow so we're spreading it over multiple processors.

**Note:** you will get a string of deprecation errors, you can safely ignore them for now.

In [155]:
print(">>> Data is all ready, now extracting features from the songs...")

# =====
# Parallel version:
#with multiprocessing.Pool(processes=num_workers,maxtasksperchild=1) as pool:
#    res=pool.starmap(get_features_mean,zip(song_data,song_sr,itertools.repeat(hop_length1),itertools.repeat(n_fft1)))
#    pool.close()
#    pool.join()

# Serial version:
res=[]
for i in range(0,len(song_data)):
    res.append(get_features_mean(song_data[i], song_sr[i], hop_length1, n_fft1))

# =====

print('>>> extracted {0} features from all songs'.format(len(res)))

>>> Data is all ready, now extracting features from the songs...
>>> extracting features...


<ipython-input-137-a6ffc4aa0dc4>:56: FutureWarning: Pass y=[ 0.00078168  0.00011291 -0.00219711 ...  0.0024626   0.00671026
  0.        ], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  centroid=lb.feature.spectral_centroid(song, sr, n_fft=n_fft, hop_length=hop_length)	#Compute the spectral centroid.
<ipython-input-137-a6ffc4aa0dc4>:59: FutureWarning: Pass y=[ 0.00078168  0.00011291 -0.00219711 ...  0.0024626   0.00671026
  0.        ], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  bw=lb.feature.spectral_bandwidth(song, sr, n_fft=n_fft, hop_length=hop_length)	#Compute p’th-order spectral bandwidth:
<ipython-input-137-a6ffc4aa0dc4>:62: FutureWarning: Pass y=[ 0.00078168  0.00011291 -0.00219711 ...  0.0024626   0.00671026
  0.        ], sr=22050 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  contrast=lb.feature.spec

>>> features extracted successfully
>>> extracted 1 features from all songs


Once we've got the features we concatenate them all into a single dictionary:

In [156]:
# concatenate each songs features (res) into dictionary
print('>>> concatenating results into a massive dictionary...')

data_dict_mean={}
for i in range(0,len(songdb)):
    data_dict_mean.update({song_name[i]:res[i]})

>>> concatenating results into a massive dictionary...


In [157]:
print(data_dict_mean)

{'AMBForst_Forest (ID 0100)_BSB.wav': {'rmseP_a': 0.11515416, 'rmseP_std': 0.11108603, 'rmseH_a': 0.06898209, 'rmseH_std': 0.10873733, 'centroid_a': 2670.9216336968484, 'centroid_std': 761.9857904747197, 'bw_a': 2236.4312818223993, 'bw_std': 290.3332567420034, 'contrast_a': 18.25936548874523, 'contrast_std': 8.344514563891384, 'polyfeat_a': -1.2968211861009747e-05, 'polyfeat_std': 4.936520110586135e-06, 'tonnetz_a': -0.0030104394795337115, 'tonnetz_std': 0.07182264646223933, 'zcr_a': 0.19776524688855185, 'zcr_std': 0.10311405333884556, 'onset_a': 1.2852021, 'onset_std': 0.5186931, 'bpm': 123.046875, 'rmseP_skew': array([0., 0., 0., 0., 0., 0.], dtype=float32), 'rmseP_kurtosis': array([-3., -3., -3., -3., -3., -3.], dtype=float32), 'rmseH_skew': array([0., 0., 0., 0., 0., 0.], dtype=float32), 'rmseH_kurtosis': array([-3., -3., -3., -3., -3., -3.], dtype=float32), '5band_rms_h0': 0.46638286, '5band_rms_p0': 0.30210438, '5band_std_h0': 0.2728199, '5band_std_p0': 0.07951982, '5band_skew_h0

...and that's it, really. Now we can just check our features and save them for use with our machine learning algorithms later.

In [158]:
# print features to screen to check
print('>>> The features extracted from the songs are: ')
print(res[0].keys())

print("----------")
print('>>> Saving dictionary to disk...')
save_obj(data_dict_mean,savefile)

end_feat=time.time() # note finish time
print("----------")
print("loading time: {0} seconds".format(start_feat-start_load))
print("feature extraction time: {0} seconds".format(end_feat-start_feat))
print("total time: {0} seconds".format(end_feat-start_load))
print("----------")
print('>>> Finished')

>>> The features extracted from the songs are: 
dict_keys(['rmseP_a', 'rmseP_std', 'rmseH_a', 'rmseH_std', 'centroid_a', 'centroid_std', 'bw_a', 'bw_std', 'contrast_a', 'contrast_std', 'polyfeat_a', 'polyfeat_std', 'tonnetz_a', 'tonnetz_std', 'zcr_a', 'zcr_std', 'onset_a', 'onset_std', 'bpm', 'rmseP_skew', 'rmseP_kurtosis', 'rmseH_skew', 'rmseH_kurtosis', '5band_rms_h0', '5band_rms_p0', '5band_std_h0', '5band_std_p0', '5band_skew_h0', '5band_skew_p0', '5band_kurtosis_h0', '5band_kurtosis_p0', '5band_rms_h1', '5band_rms_p1', '5band_std_h1', '5band_std_p1', '5band_skew_h1', '5band_skew_p1', '5band_kurtosis_h1', '5band_kurtosis_p1', '5band_rms_h2', '5band_rms_p2', '5band_std_h2', '5band_std_p2', '5band_skew_h2', '5band_skew_p2', '5band_kurtosis_h2', '5band_kurtosis_p2', '5band_rms_h3', '5band_rms_p3', '5band_std_h3', '5band_std_p3', '5band_skew_h3', '5band_skew_p3', '5band_kurtosis_h3', '5band_kurtosis_p3'])
----------
>>> Saving dictionary to disk...
----------
loading time: 2.7696914672

1- add more music to the "free_music" folder, and run this notebook again.